In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc

2024-02-23 00:45:29.342996: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 00:45:34.581995: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
pip freeze

absl-py==1.4.0
accelerate @ file:///build/jenkins/workspace/lcg_release_pipeline/build/pyexternals/accelerate-0.20.3/src/accelerate/0.20.3
AGILe==1.5.0
aiohttp==3.8.4
aiosignal==1.2.0
aiostream @ file:///build/jenkins/workspace/lcg_release_pipeline/build/pyexternals/aiostream-0.4.5/src/aiostream/0.4.5
alabaster==0.7.12
altair @ file:///build/jenkins/workspace/lcg_release_pipeline/build/pyexternals/altair-5.2.0/src/altair/5.2.0
anyio @ file:///build/jenkins/workspace/lcg_release_pipeline/build/pyexternals/anyio-3.7.1/src/anyio/3.7.1
appdirs==1.4.4
archspec==0.1.4
argon2-cffi==20.1.0
asn1crypto==0.24.0
asteval @ file:///build/jenkins/workspace/lcg_release_pipeline/build/pyexternals/asteval-0.9.31/src/asteval/0.9.31
astor==0.8.1
astroid @ file:///build/jenkins/workspace/lcg_release_pipeline/build/pyexternals/astroid-2.15.5/src/astroid/2.15.5
astropy==5.3
astunparse==1.6.3
async-generator==1.10
async-timeout==4.0.2
atomicwrites==1.3.0
attrs @ file:///build/jenkins/workspace/lcg_release_pip

In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

truthtable=[]

threshold=50
for i, batch in enumerate(Dataset):
  if np.sum(batch[:,0])>=threshold:
    truthtable+=[1]
  else:
    truthtable+=[0]

event_pt_br=[]
Data_Test_full=Dataset[2000001:3600000,:,:]
for j, br_1 in enumerate(Data_Test_full):
  event_pt_br+=[np.sum(br_1[:,0])]

for i, batch in enumerate(Dataset):
  pt_sum=0
  for j, particle in enumerate(Dataset[i,:,:]):
    if particle[3]!=0:
      pt_sum+=particle[0]
  for j, particle in enumerate(Dataset[i,:,:]):
    particle[0]=particle[0]/pt_sum

    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Test_Truth=truthtable[2000001:3600000]
Data_Validate=Dataset[3600001:4000000,:,0:3]

In [3]:
def create_AE(input_dim, h_dim_1, h_dim_2, latent_dim):
    # Encoder
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(h_dim_1, activation='relu')(inputs)
    x = layers.Dense(h_dim_2, activation='relu')(x)
    z = layers.Dense(latent_dim, activation='relu')(x)

    # Decoder
    x = layers.Dense(h_dim_2, activation='relu')(z)
    x = layers.Dense(h_dim_1, activation='relu')(x)
    outputs = layers.Dense(input_dim)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
def loss_fn(y_true, y_pred):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    return K.mean(squared_difference)

In [5]:
Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [6]:

INPUT_DIM = Data_Train_Flat.shape[1]
H_DIM_1 = 32
H_DIM_2 = 16
LATENT_DIM = 8
DNN_AE = create_AE(input_dim=INPUT_DIM, h_dim_1=H_DIM_1, h_dim_2=H_DIM_2, latent_dim=LATENT_DIM)
DNN_AE.summary()

2024-01-26 22:42:08.410020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-26 22:42:08.598026: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-26 22:42:08.598449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 57)]              0         
                                                                 
 dense (Dense)               (None, 32)                1856      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 16)                144       
                                                                 
 dense_4 (Dense)             (None, 32)                544       
                                                                 
 dense_5 (Dense)             (None, 57)                1881  

In [7]:
DNN_AE.compile(optimizer='adam', loss=loss_fn)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 50
BATCH_SIZE = 1024

history = DNN_AE.fit(x=Data_Train_Flat, y=Data_Train_Flat, validation_data=(Data_Val_Flat,Data_Val_Flat), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

Epoch 1/50


2024-01-26 22:42:29.430987: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f89f800c730 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-26 22:42:29.431061: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-01-26 22:42:29.519699: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-26 22:42:31.559394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-26 22:42:32.728213: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1954/1954 [==============================] - 28s 5ms/step - loss: 0.0234 - val_loss: 0.0077 - lr: 0.0010
Epoch 2/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0073 - val_loss: 0.0068 - lr: 0.0010
Epoch 3/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0067 - val_loss: 0.0064 - lr: 0.0010
Epoch 4/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0064 - val_loss: 0.0062 - lr: 0.0010
Epoch 5/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0062 - val_loss: 0.0061 - lr: 0.0010
Epoch 6/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0061 - val_loss: 0.0059 - lr: 0.0010
Epoch 7/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0060 - val_loss: 0.0058 - lr: 0.0010
Epoch 8/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0059 - val_loss: 0.0058 - lr: 0.0010
Epoch 9/50
1954/1954 [==============================] - 9s 5ms/step - loss: 0.0057 - val_l

In [8]:
DNN_AE.save('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/40MHZ_norm_DNN.keras')